In [1]:
import numpy as np
from scipy.optimize import linprog
from ipynb.fs.full.functions import nn2na, get_selected_arcs, arc_usage, get_min_cut

In [2]:
#Import data
NN = np.array([[0, 1, 1, 0, 0, 0],
               [0, 0, 0, 1, 0, 1],
               [0, 0, 0, 0, 1, 0],
               [0, 0, 0, 0, 0, 1],
               [0, 0, 0, 0, 0, 1],
               [1, 0, 0, 0, 0, 0]])
beq = np.array([0, 0, 0, 0, 0, 0])
# Every arc has a cost of zero but the t->s arc has a cost of -1 in order to encourage the flow to go through it:
C = np.array([0, 0, 0, 0, 0, 0, 0, -1])
# Upper bounds (capacity) of each arc flow: 
upper_b = [7, 1, 2, 3, 2, 1, 2, None] # t->s arc has unlimitted capacity

In [3]:
# DATA transforming for optimization:
Aeq, arc_idxs,arc_idxs_list = nn2na(NN)
bounds = tuple([(0, upper_b[arcs]) for arcs in range(0, Aeq.shape[1])])

In [4]:
# Resulting bounds tuple:
print(bounds)

((0, 7), (0, 1), (0, 2), (0, 3), (0, 2), (0, 1), (0, 2), (0, None))


In [5]:
# OPTIMIZE:
res = linprog(C, A_eq=Aeq, b_eq=beq, bounds=bounds, method='simplex')
res.message

C:\Users\joaquin\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: OptimizeWarning: A_eq does not appear to be of full row rank. To improve performance, check the problem formulation for redundant equality constraints.
  


'Optimization terminated successfully.'

In [6]:
# Results equations
arc_use = arc_usage(arc_idxs, res.x.astype(int)) # Function to bring flow and indexes of each arc
result = res.fun*(-1)
mincut = get_min_cut(arc_idxs, res.x, np.array(upper_b))

In [7]:
print('### Results of Max Flow ### \n'
      'The max flow circulating across the system is: %s \n'
      'The flow circulating through each arc is: %s \n'
      'The arcs that form the minimum cut are: %s'%(result, arc_use, mincut))

### Results of Max Flow ### 
The max flow circulating across the system is: 5.0 
The flow circulating through each arc is: {(0, 1): 4, (0, 2): 1, (1, 3): 1, (1, 5): 3, (2, 4): 1, (3, 5): 1, (4, 5): 1, (5, 0): 5} 
The arcs that form the minimum cut are: [(0, 2), (1, 5), (3, 5)]


### CONCLUSION
From t->s the flow is 5. So that is the maximum flow circulating through the system given those capacities. The flow through the arcs of minimum cut add up to 5. It represents the dual of the original problem. That is because in the optimal point, the maximum of the primal is equal to the minimum of the dual.